In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [18]:
chrome_options = Options()
chrome_servis = Service('chromedriver.exe')

In [19]:
driver = webdriver.Chrome(service=chrome_servis, options=chrome_options)
# chrome_options.add_argument("--headless")

In [20]:
# Page 1
url = "https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanbod/"
driver.set_window_size(1300, 800)
driver.get(url)
time.sleep(5)

In [21]:
driver.save_screenshot("home.png")
content = driver.page_source
driver.quit()

In [22]:
# Soup Object containing all data in Page 1
page_soup = BeautifulSoup(content, 'html.parser')

#Creating Lists
manufacturer_list, model_list, price_list, carrosserie_list, fuel_list = [],[],[],[],[]

In [23]:
i = 1
car_info = {}
for data in page_soup.find_all('div', class_='sc-bLESgL bplabz'):
    print(i)
    name_span = data.find('span', class_='sc-fQgSAe duLMLp sc-eODrEC cXeHSQ PONCHO-typography PONCHO-typography--button-link')
    
    #Check if the span element is found
    if name_span:
        text_content = name_span.get_text(strip=True)
        manufacturer, model = text_content.split(' ', 1)
        print('Manufacturer: ', manufacturer)
        print('Model: ', model)
    else:
        print('Name information not found.')
    price = data.find('span', class_='sc-Gppvi kvInjJ PONCHO-typography PONCHO-typography--price-text-xl').get_text()
    i+=1
    print('Price: ', price)

    #initiate the new_soup
    links = data.find('a', class_='sc-cDJyZ vNUGH')
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_servis = Service('chromedriver.exe')
    driver2 = webdriver.Chrome(service=chrome_servis, options=chrome_options)
    driver2.get(links.get('href'))
    new_content = driver2.page_source
        
    new_soup = BeautifulSoup(new_content, 'html.parser')
    
    ul_element = new_soup.find('ul', class_='sc-daa54801-0 dBGuIu PONCHO-list PONCHO-list--bulleted')
    if ul_element:
        for datas in ul_element:
            text_content2 = datas.get_text(strip=True)

            if 'Carrosserie:' in text_content2:
                car_info['Carrosserie'] = text_content2.split(':', 1)[1].strip()
            elif 'Brandstof:' in text_content2:
                car_info['Brandstof'] = text_content2.split(':', 1)[1].strip()

                print('Carrosserie:', car_info.get('Carrosserie'))
                print('Fuel:', car_info.get('Brandstof'))
    
        driver2.quit()

        # manufacturer_list.append(manufacturer)
        # model_list.append(model)
        # price_list.append(price)
        # carrosserie_list.append(car_info.get('Carrosserie'))
        # fuel_list.append(car_info.get('Brandstof'))

    else:
        print("Element not found.")
        car_info['Carrosserie'] = '-'
        car_info['Brandstof'] = '-'
    
    manufacturer_list.append(manufacturer)
    model_list.append(model)
    price_list.append(price)
    carrosserie_list.append(car_info.get('Carrosserie'))
    fuel_list.append(car_info.get('Brandstof'))

    print("-------")

1
Manufacturer:  Hyundai
Model:  i10
Price:  € 249,-
Carrosserie: Hatchback (5 deurs)
Fuel: Benzine
-------
2
Manufacturer:  Hyundai
Model:  i20 MHEV
Price:  € 319,-
Carrosserie: Hatchback (5 deurs)
Fuel: Benzine
-------
3
Manufacturer:  Hyundai
Model:  Bayon MHEV
Price:  € 349,-
Carrosserie: SUV (5 deurs)
Fuel: Benzine
-------
4
Manufacturer:  Hyundai
Model:  IONIQ 5 Elektrisch
Price:  € 539,-
Carrosserie: Hatchback (5 deurs)
Fuel: Elektriciteit
-------
5
Manufacturer:  Hyundai
Model:  IONIQ 6 Elektrisch
Price:  € 569,-
Carrosserie: Sedan (5 deurs)
Fuel: Elektriciteit
-------
6
Manufacturer:  Hyundai
Model:  Tucson Hybrid
Price:  € 599,-
Carrosserie: SUV (5 deurs)
Fuel: Hybride benzine
-------
7
Manufacturer:  Mitsubishi
Model:  Space Star
Price:  € 279,-
Carrosserie: Hatchback (5 deurs)
Fuel: Benzine
-------
8
Manufacturer:  Suzuki
Model:  Ignis MHEV
Price:  € 279,-
Carrosserie: SUV (5 deurs)
Fuel: Benzine
-------
9
Manufacturer:  Toyota
Model:  AYGO X
Price:  € 299,-
Carrosserie: Ha

In [26]:
df = pd.DataFrame({'Manufacturer':manufacturer_list,'Model':model_list,'Price':price_list,'Carrosserie':carrosserie_list,'Fuel':fuel_list})
df.to_csv("leasing_car.csv", header=True, index=False)

with pd.ExcelWriter('leasing_car.xlsx', engine='openpyxl') as writer:
  df.to_excel(writer, sheet_name='Sheet1', index=False)